In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import optimize
import time

In [2]:
dataframe = pd.read_csv("./FATS_GAIA.dat")
dataframe = dataframe.drop(["Class", "ID"], axis=1)
dataframe.head()

,Amplitude,AndersonDarling,Autocor_length,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,FluxPercentileRatioMid80,...,PeriodLS,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,1.416432,0.949482,3.0,0.000000,5.442293,0.243054,0.462373,0.653511,0.761790,0.924793,...,268.331543,0.025528,0.304804,0.310355,1.784879,0.304804,0.011895,19.899170,-1.266215,0.931676
1,0.443386,1.000000,1.0,0.000000,12042.752633,0.162446,0.231268,0.517775,0.720122,0.836798,...,0.508910,0.012390,0.309458,0.519645,0.397205,0.258203,-0.962335,0.147949,-0.183573,0.284637
2,0.170099,0.875986,4.0,0.030303,137.020266,0.033964,0.208724,0.378623,0.501416,0.869956,...,8.742769,0.040022,0.342646,0.700200,0.119849,0.368936,0.472161,0.147949,0.064528,0.089020
3,1.350858,0.999869,4.0,0.000000,23.583559,0.259476,0.593270,0.613808,0.817245,0.943033,...,379.949707,0.000843,0.349687,0.285682,1.867060,0.349687,-0.047072,23.597900,-1.454570,0.926506
4,0.248472,0.999947,3.0,0.000000,125.486491,0.202240,0.448444,0.565120,0.606539,0.840512,...,318.427795,0.022169,0.247451,0.578087,0.273482,0.247451,0.594605,0.147949,-0.689124,0.153349


## Preprocesamiento: dividir base de datos en *train* y *test*

In [3]:
train = dataframe.sample(frac=0.8, random_state=7).reset_index()
train = train.drop("index", 1)
test = dataframe.drop(train.index).reset_index()
test = test.drop("index", 1)

In [4]:
X_train = train[[column for column in train.columns if column != "PeriodLS"]]
y_train = train["PeriodLS"]
X_test = test[[column for column in train.columns if column != "PeriodLS"]]
y_test = test["PeriodLS"]

## Construcción del árbol

In [5]:
class Node:
    id_count = 0

    def __init__(self, X, y, parent=None):
        self.X = X
        self.y = y
        self.parent = parent
        self.id = Node.id_count
        self.left = None
        self.right = None
        Node.id_count += 1
        self.children = dict()

    def find_best_split(self):
        total_node_variance = self.y.var()
        features_var_red = {}
        for feature in self.X.columns:
            print(feature)
            feature_values = self.X[feature].unique()
            values_var_red = []
            for value in feature_values:
                left_side = self.X[self.X[feature] <= value]
                right_side = self.X[self.X[feature] > value]
                left_side_var = self.y[self.y.index.isin(left_side.index)]
                right_side_var = self.y[self.y.index.isin(right_side.index)]
                final_var = total_node_variance - \
                    (left_side_var + right_side_var)
                values_var_red.append(final_var)
            max_var_red = max(values_var_red)
            features_var_red[feature] = max_var_red
        return max(features_var_red, key=features_var_red.get)

    def split_node(self):
        left_data = self.X[self.X[self.split_label] <= self.split_value]
        right_data = self.X[self.X[self.split_label] > self.split_value]
        left_target = self.y[self.y.index.isin(left_data.index)]
        right_target = self.y[self.y.index.isin(right_data.index)]
        return left_data, left_target, right_data, right_target

    @staticmethod
    def var_red_function(value, feature, X_values, y_values):
        left_side = X_values[X_values[feature] <= value]
        right_side = X_values[X_values[feature] > value]
        left_side_y = y_values[y_values.index.isin(left_side.index)]
        right_side_y = y_values[y_values.index.isin(right_side.index)]
        return left_side_y.var() + right_side_y.var()

    @staticmethod
    def calculate_rss(y_values):
        return np.sum((y_values - y_values.mean())**2)

    @staticmethod
    def rss_function(value, feature, X_values, y_values):
        left_side = X_values[X_values[feature] <= value]
        right_side = X_values[X_values[feature] > value]
        left_side_y = y_values[y_values.index.isin(left_side.index)]
        right_side_y = y_values[y_values.index.isin(right_side.index)]
        return Node.calculate_rss(left_side_y) + Node.calculate_rss(
            right_side_y)

    def find_split_scipy(self, mode="rss"):
        features_var_red = dict()
        for feature in self.X.columns:
            min_feature_value = self.X[feature].min()
            max_feature_value = self.X[feature].max()
            if min_feature_value == max_feature_value:
                continue
            if mode == "rss":
                optimal_split, split_var_red, ierr, numf = scipy.optimize.fminbound(
                    Node.rss_function,
                    min_feature_value,
                    max_feature_value,
                    args=(feature, self.X, self.y),
                    full_output=1)
            else:
                optimal_split, split_var_red, ierr, numf = scipy.optimize.fminbound(
                    Node.var_red_function,
                    min_feature_value,
                    max_feature_value,
                    args=(feature, self.X, self.y),
                    full_output=1)                
            features_var_red[feature] = (split_var_red, optimal_split)
        best_split = sorted(features_var_red.items(), key=lambda x: x[1])[0]
        self.split_label = best_split[0]
        self.split_value = best_split[1][-1]
        return self.split_label, self.split_value

In [6]:
class RegressionTree:
    def __init__(self, max_depth=5, min_values=5, optimize="rss"):
        self.max_depth = max_depth
        self.min_values = min_values
        self.optimize = optimize
        self.depth = 0

    def fit(self, data, target, parent=None, depth=0):
        root = Node(data, target)
        if len(root.X) <= self.min_values:
            return root
        if depth >= self.max_depth:
            return root
        root.find_split_scipy(mode=self.optimize)
        left_data, left_target, right_data, right_target = root.split_node()
        root.left = self.fit(left_data, left_target, root, depth + 1)
        root.right = self.fit(right_data, left_target, root, depth + 1)
        self.root = root

    @staticmethod
    def predict_datum(root, datum):
        if root.left is None:
            return root.y.mean()
        if datum.loc[root.split_label] <= root.split_value:
            return predict_datum(root.left, datum)
        return RegressionTree.predict_datum(root.right, datum)

    def predict(self, test_data):
        return test_data.apply(lambda x: RegressionTree.predict_datum(self.root, x), axis=1)

In [7]:
def calculate_rmse(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())

In [8]:
tree_rss = RegressionTree(5, 5, "rss")
fit_start_time_rss = time.time()
tree_rss.fit(X_train, y_train)
fit_rss_time = time.time() - fit_start_time_rss
print(fit_rss_time)

396.30714893341064


In [9]:
y_pred_rss = tree_rss.predict(X_test)
rmse_rss = calculate_rmse(y_pred_rss, y_test)
print(rmse_rss)

179.29570279623692


In [10]:
tree_var_red = RegressionTree(5, 5, "var_red")
fit_start_time_var_red = time.time()
tree_var_red.fit(X_train, y_train)
fit_var_red_time = time.time() - fit_start_time_var_red
print(fit_var_red_time)

358.44494795799255


In [11]:
y_pred_var_red = tree_var_red.predict(X_test)
rmse_var_red = calculate_rmse(y_pred_var_red, y_test)
print(rmse_var_red)

179.29570279623692


In [12]:
print(abs(rmse_rss - rmse_var_red))

0.0
